In [43]:
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

In [44]:
def extract_features_with_timing(file_path, start_time, end_time):
    audio, sample_rate = librosa.load(file_path, offset=start_time, duration=end_time - start_time, res_type='kaiser_fast')
    features = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
    return features


In [45]:
# Predict Class
def predict_calsses_for_audio(file_path, model_path, class_path):
    model = keras.models.load_model(model_path)
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load(class_path, allow_pickle=True)

    #Ekstrak MFCC
    features = extract_features_with_timing(file_path, start_time=0, end_time=librosa.get_duration(path=file_path))
    features = features.reshape(1, -1)

    predictions = model.predict(features)

    total_duration = librosa.get_duration(path=file_path)
    class_percentages = {class_label: percentage * 100 for class_label, percentage in zip(label_encoder.classes_, predictions[0])}

    return class_percentages

In [46]:
audio_path = "DATA_NIGENS\engine\AMBULANCE_GEN-HDF-00814.wav"

model_path = "testModel.keras"
class_path = "label_encoderTest.joblib"
class_percentages = predict_calsses_for_audio(audio_path, model_path, class_path)

highest_class_label = None
highest_percentage = 0

for class_label, percentage in class_percentages.items():
    print(f"Class: {class_label}, Percentage: {percentage: .2f}%")

    if percentage > highest_percentage:
        highest_percentage = percentage
        highest_class_label = class_label
if highest_class_label:
    print(f"\nThere is high  possibilities this is a {highest_class_label} sound with percentage {highest_percentage: .2f}%")

1/1 [==============================] - 0s 81ms/step
Class: alarm, Percentage:  1.07%
Class: baby, Percentage:  0.00%
Class: crash, Percentage:  0.87%
Class: dog, Percentage:  0.01%
Class: engine, Percentage:  70.75%
Class: femaleScream, Percentage:  0.00%
Class: femaleSpeech, Percentage:  0.13%
Class: fire, Percentage:  0.09%
Class: footsteps, Percentage:  0.10%
Class: general, Percentage:  20.48%
Class: knock, Percentage:  4.55%
Class: maleScream, Percentage:  0.00%
Class: maleSpeech, Percentage:  0.04%
Class: phone, Percentage:  0.00%
Class: piano, Percentage:  1.90%

There is high  possibilities this is a engine sound with percentage  70.75%
